In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import missingno as msno
from pathlib import Path
from tqdm import tqdm
import wandb
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
train=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold
import tensorflow as tf

In [ ]:
train["file_path"] = train["Id"].apply(lambda x: "../input/petfinder-pawpularity-score/train/" + x + ".jpg")
test["file_path"] = test["Id"].apply(lambda x: "../input/petfinder-pawpularity-score/test/" + x + ".jpg")

In [ ]:
image_size = 128
def preprocess(image_url):
    image_string = tf.io.read_file(image_url)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.central_crop(image, 1.0)
    image = tf.image.resize(image, (image_size, image_size))
    return image
x_train=[]
for i in train['file_path']:
    x1=preprocess(i)
    x_train.append(x1)

In [ ]:
test2=[]
for i in test['file_path']:
    x1=preprocess(i)
    test2.append(x1)
test2=np.array(test2)

In [ ]:
X=np.array(x_train)
y=train['Pawpularity']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=86)

In [ ]:
def create_model(input_shape, top='flatten'):
    if top not in ('flatten', 'avg', 'max'):
        raise ValueError('unexpected top layer type: %s' % top)

    # connects base model with new "head"
    BottleneckLayer = {
        'flatten': keras.layers.Flatten(),
        'avg': keras.layers.GlobalAveragePooling2D(),
        'max': keras.layers.GlobalMaxPool2D()
    }[top]

    base = tf.keras.applications.ResNet50(input_shape=input_shape,
                             include_top=False, 
                             weights=None)

    x = BottleneckLayer(base.output)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(1, activation='relu')(x)
    model = tf.keras.Model(inputs=base.inputs, outputs=x)
    return model
input_shape=(128,128,3)
model=create_model(input_shape)

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_rmse',patience=10, verbose=1,mode='min',restore_best_weights=True
    )
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_rmse",
    factor=0.2,
    patience=3, 
    min_lr=1e-7,
    verbose=1,
    mode='min'
    )
callbacks = [early_stop,reduce_lr]

In [ ]:
model.compile(loss='mse', optimizer='Adam', metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

In [ ]:
history = model.fit(x_train,y_train,epochs=100,batch_size=128,validation_data=(x_test,y_test),callbacks=callbacks)

In [ ]:
cnn_pred=model.predict(test2)

In [ ]:
cnn=pd.DataFrame()
cnn['Id']=test['Id']
cnn['Pawpularity']=cnn_pred
cnn.to_csv('submission.csv',index=False)

In [ ]:
cnn